In [47]:
%load_ext autoreload
%autoreload 2

import modified_convcode as mcc
import turbo
import modified_turbo as mt
from commpy import channelcoding as cc
import numpy as np
from commpy.utilities import dec2bitarray, bitarray2dec

SEED = 0
def get_test_rng(seed=SEED):
    return np.random.default_rng(seed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
block_len = 5

test_message_bits = np.array([1, 1, 0, 1, 0])


In [8]:
cc_gen = np.array([[7, 5]])
cc_M = np.array([2])
cc_trellis = cc.Trellis(cc_M, cc_gen, feedback=0)
cc_interleaver = cc.RandInterlv(block_len, 0)

print("cc_trellis")
print(f"Trellis is code type {cc_trellis.code_type} and rate {cc_trellis.k}/{cc_trellis.n}")
print("Output table")
display(cc_trellis.output_table)
print("Next State table")
display(cc_trellis.next_state_table)

cc_trellis
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [3, 0],
       [2, 1],
       [1, 2]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

In [9]:
mcc_gen = np.array([[1, 1, 1], [1, 0 , 1]])
mcc_bias = np.array([0, 0])
mcc_trellis = mcc.ModifiedTrellis(mcc_gen,mcc_bias)
mcc_interleaver = cc.RandInterlv(block_len, 0)

print("mcc_trellis")
print(f"Trellis is code type {mcc_trellis.code_type} and rate {mcc_trellis.k}/{mcc_trellis.n}")
print("Output table")
display(mcc_trellis.output_table)
print("Next State table")
display(mcc_trellis.next_state_table)

mcc_trellis
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [3, 0],
       [2, 1],
       [1, 2]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

In [10]:
[cc_s1, cc_s2, cc_s3] = cc.turbo_encode(test_message_bits, cc_trellis, cc_trellis, cc.RandInterlv(block_len, 0))
print("stream 1")
display(cc_s1)
print(f"Stream 1 is same as message (systematuic? {np.all(test_message_bits == cc_s1)}")
display(cc_s2)
display(cc_s3)

encoding turbo code
Enterted conv encode with k=1; n=2; total_memory=2; rate=0.5; code_type=default
Using termination rsc
Running term termination
Running default code type
number_inbit=7; inbits=[1 1 0 1 0 0 0]; number_outbits=14
Do I ever reach this code????
Enterted conv encode with k=1; n=2; total_memory=2; rate=0.5; code_type=default
Using termination rsc
Running term termination
Running default code type
number_inbit=7; inbits=[0 1 1 1 0 0 0]; number_outbits=14
Do I ever reach this code????
stream 1


array([1, 0, 0, 0, 1])

Stream 1 is same as message (systematuic? False


array([1, 1, 1, 0, 0])

array([0, 1, 1, 0, 1])

In [11]:
[mcc_s1, mcc_s2, mcc_s3] = mt.turbo_encode(test_message_bits, mcc_trellis, mcc_trellis, cc.RandInterlv(block_len, 0))
print("My stream 1")
display(mcc_s1)
print(f"Stream 1 is same as message? {np.all(test_message_bits == mcc_s1)}")
display(mcc_s2)
display(mcc_s3)

encoding turbo code
[MCC] Modfied msg bits: [1 1 0 1 0]
[MCC] Num inbits 5
[MCC] Num outbits 10
[MCC] Modfied msg bits: [0 1 1 1 0]
[MCC] Num inbits 5
[MCC] Num outbits 10
My stream 1


array([1, 0, 0, 0, 1])

Stream 1 is same as message? False


array([1, 1, 1, 0, 0])

array([0, 1, 1, 0, 1])

In [15]:
from utils import get_test_sigmas

test_sigmas = get_test_sigmas(-1.5, 2.0, 8)
sigma = 1

[testing] SNR range in dB  [-1.5 -1.  -0.5  0.   0.5  1.   1.5  2. ]
[testing] Test sigmas are [1.18850223 1.12201845 1.05925373 1.         0.94406088 0.89125094
 0.84139514 0.79432823]
[sanity check]: SNRs for sigmas are [-1.5000000000000002, -1.0000000000000009, -0.49999999999999956, -0.0, 0.49999999999999983, 0.9999999999999993, 1.4999999999999991, 1.9999999999999998]


In [16]:
from utils import awgn_corrupt

cc_rng = get_test_rng()

cc_s1_r = awgn_corrupt(cc_rng, cc_s1, sigma = sigma)
cc_s2_r = awgn_corrupt(cc_rng, cc_s2, sigma = sigma)
cc_s3_r = awgn_corrupt(cc_rng, cc_s3, sigma = sigma)

display(cc_s1_r)
display(cc_s2_r)
display(cc_s3_r)

array([ 1.12573022, -1.13210486, -0.35957735, -0.89509988,  0.46433063])

array([ 1.36159505,  2.30400005,  1.94708096, -1.70373524, -2.26542147])

array([-1.62327446,  1.04132598, -1.32503077, -1.21879166, -0.24591095])

In [17]:
mcc_rng = get_test_rng()

mcc_s1_r = awgn_corrupt(mcc_rng, mcc_s1, sigma = sigma)
mcc_s2_r = awgn_corrupt(mcc_rng, mcc_s2, sigma = sigma)
mcc_s3_r = awgn_corrupt(mcc_rng, mcc_s3, sigma = sigma)

display(mcc_s1_r)
display(mcc_s2_r)
display(mcc_s3_r)

array([ 1.12573022, -1.13210486, -0.35957735, -0.89509988,  0.46433063])

array([ 1.36159505,  2.30400005,  1.94708096, -1.70373524, -2.26542147])

array([-1.62327446,  1.04132598, -1.32503077, -1.21879166, -0.24591095])

In [24]:
cc_res = turbo.hazzys_turbo_decode(cc_s1_r, cc_s2_r, cc_s3_r, cc_trellis, sigma ** 2, 6, cc_interleaver)
cc_res.astype(int)

array([1, 1, 0, 1, 0])

In [25]:
mcc_res = turbo.hazzys_turbo_decode(mcc_s1_r, mcc_s2_r, mcc_s3_r, mcc_trellis, sigma ** 2, 6, mcc_interleaver)
mcc_res.astype(int)

array([1, 1, 0, 1, 0])

In [43]:
# Now try turbo code with different trellises
cc_gen1 = np.array([[4, 7]])
cc_gen2 = np.array([[4, 5]])
cc_M = np.array([2])
cc_trellis1 = cc.Trellis(cc_M, cc_gen1, feedback=0)
cc_trellis2 = cc.Trellis(cc_M, cc_gen2, feedback=0)
cc_test2_interleaver = cc.RandInterlv(block_len, 0)

print("cc_trellis1")
print(f"Trellis is code type {cc_trellis1.code_type} and rate {cc_trellis1.k}/{cc_trellis1.n}")
print("Output table")
display(cc_trellis1.output_table)
print("Next State table")
display(cc_trellis1.next_state_table)

print("cc_trellis2")
print(f"Trellis is code type {cc_trellis2.code_type} and rate {cc_trellis2.k}/{cc_trellis2.n}")
print("Output table")
display(cc_trellis2.output_table)
print("Next State table")
display(cc_trellis2.next_state_table)

cc_trellis1
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [1, 2],
       [1, 2],
       [0, 3]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

cc_trellis2
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [1, 2],
       [0, 3],
       [1, 2]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

In [42]:
mcc_gen1 = np.array([[1, 0, 0], [1, 1, 1]])
mcc_gen2 = np.array([[1, 0, 0], [1, 0 , 1]])
mcc_bias = np.array([0, 0])
mcc_trellis1 = mcc.ModifiedTrellis(mcc_gen1,mcc_bias)
mcc_trellis2 = mcc.ModifiedTrellis(mcc_gen2,mcc_bias)
mcc_test2_interleaver = cc.RandInterlv(block_len, 0)

print("mcc_trellis1")
print(f"Trellis is code type {mcc_trellis1.code_type} and rate {mcc_trellis1.k}/{mcc_trellis1.n}")
print("Output table")
display(mcc_trellis1.output_table)
print("Next State table")
display(mcc_trellis1.next_state_table)

print("mcc_trellis2")
print(f"Trellis is code type {mcc_trellis2.code_type} and rate {mcc_trellis2.k}/{mcc_trellis2.n}")
print("Output table")
display(mcc_trellis2.output_table)
print("Next State table")
display(mcc_trellis2.next_state_table)

mcc_trellis1
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [1, 2],
       [1, 2],
       [0, 3]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

mcc_trellis2
Trellis is code type default and rate 1/2
Output table


array([[0, 3],
       [1, 2],
       [0, 3],
       [1, 2]])

Next State table


array([[0, 2],
       [0, 2],
       [1, 3],
       [1, 3]])

In [52]:
[cc_test2_s1, cc_test2_s2, cc_test2_s3] = cc.turbo_encode(test_message_bits, cc_trellis1, cc_trellis2, cc_test2_interleaver)
print("stream 1")
display(cc_test2_s1)
print(f"Stream 1 is same as message (systematuic? {np.all(test_message_bits == cc_test2_s1)}")
display(cc_test2_s2)
display(cc_test2_s3)

encoding turbo code
Enterted conv encode with k=1; n=2; total_memory=2; rate=0.5; code_type=default
Using termination rsc
Running term termination
Running default code type
number_inbit=7; inbits=[1 1 0 1 0 0 0]; number_outbits=14
Do I ever reach this code????
Enterted conv encode with k=1; n=2; total_memory=2; rate=0.5; code_type=default
Using termination rsc
Running term termination
Running default code type
number_inbit=7; inbits=[0 1 1 1 0 0 0]; number_outbits=14
Do I ever reach this code????
stream 1


array([1, 1, 0, 1, 0])

Stream 1 is same as message (systematuic? True


array([1, 0, 0, 0, 1])

array([0, 1, 1, 0, 1])

In [53]:
[mcc_test2_s1, mcc_test2_s2, mcc_test2_s3] = mt.turbo_encode(test_message_bits, mcc_trellis1, mcc_trellis2, mcc_test2_interleaver)
print("My stream 1")
display(mcc_test2_s1)
print(f"Stream 1 is same as message? {np.all(test_message_bits == mcc_test2_s1)}")
display(mcc_test2_s2)
display(mcc_test2_s3)

encoding turbo code
[MCC] Modfied msg bits: [1 1 0 1 0]
[MCC] Num inbits 5
[MCC] Num outbits 10
[MCC] Modfied msg bits: [0 1 1 1 0]
[MCC] Num inbits 5
[MCC] Num outbits 10
My stream 1


array([1, 1, 0, 1, 0])

Stream 1 is same as message? True


array([1, 0, 0, 0, 1])

array([0, 1, 1, 0, 1])

In [39]:
cc_test2_rng = get_test_rng()

cc_test2_s1_r = awgn_corrupt(cc_test2_rng, cc_test2_s1, sigma = sigma)
cc_test2_s2_r = awgn_corrupt(cc_test2_rng, cc_test2_s2, sigma = sigma)
cc_test2_s3_r = awgn_corrupt(cc_test2_rng, cc_test2_s3, sigma = sigma)

display(cc_test2_s1_r)
display(cc_test2_s2_r)
display(cc_test2_s3_r)

array([ 1.12573022,  0.86789514, -0.35957735,  1.10490012, -1.53566937])

array([ 1.36159505,  0.30400005, -0.05291904, -1.70373524, -0.26542147])

array([-1.62327446,  1.04132598, -1.32503077, -1.21879166, -0.24591095])

In [41]:
mcc_test2_rng = get_test_rng()

mcc_test2_s1_r = awgn_corrupt(mcc_test2_rng, mcc_test2_s1, sigma = sigma)
mcc_test2_s2_r = awgn_corrupt(mcc_test2_rng, mcc_test2_s2, sigma = sigma)
mcc_test2_s3_r = awgn_corrupt(mcc_test2_rng, mcc_test2_s3, sigma = sigma)

display(mcc_test2_s1_r)
display(mcc_test2_s2_r)
display(mcc_test2_s3_r)

array([ 1.12573022,  0.86789514, -0.35957735,  1.10490012, -1.53566937])

array([ 1.36159505,  0.30400005, -0.05291904, -1.70373524, -0.26542147])

array([-1.62327446,  1.04132598, -1.32503077, -1.21879166, -0.24591095])

In [50]:
cc_test2_res = mt.hazzys_turbo_decode(cc_test2_s1_r, cc_test2_s2_r, cc_test2_s3_r, cc_trellis1, cc_trellis2, sigma ** 2, 6, cc_test2_interleaver)
cc_test2_res.astype(int)

array([1, 1, 0, 1, 0])

In [51]:
mcc_test2_res = mt.hazzys_turbo_decode(mcc_test2_s1_r, mcc_test2_s2_r, mcc_test2_s3_r, mcc_trellis1, mcc_trellis2, sigma ** 2, 6, mcc_test2_interleaver)
mcc_test2_res.astype(int)

array([1, 1, 0, 1, 0])